# Exercise 8: Seeing the Brain “Learn” through Reward Prediction Errors
## A quick look at the MID task

Here we move from artificial networks back to human brains, using the **Monetary Incentive Delay (MID)** task. In the MID:

1. Participants see a cue predicting potential reward, neutral outcome, or loss.  
2. They respond quickly to a target to try to earn money (or avoid losing it).  
3. Feedback tells them if they succeeded, generating a **prediction error** when outcome differs from expectation.  

> **See “Guide to Exercise 8” slides** for a step-by-step demo of the MID timing, trial types, and expected brain responses.

---

## What you will do in this notebook (and why)

### 1. **QC Anatomy**  
You will load brain images and verify the alignment of your region of interest (ROI)—the nucleus accumbens (NAcc).  
**Why?**  
✅ To make sure the mask correctly covers the NAcc (more on the mask below). If the mask is off, any measurements from it would not represent true NAcc activity.

#### Exploring the NAcc in the ABCD Data Dictionary

For a deeper understanding of the nucleus accumbens (NAcc), look up the following metrics in the ABCD data dictionary and visualize them using the Brain Atlas Visualizer:
https://abcd.deapscience.com/#/my-datasets/create-dataset

1. **mr_y_smri__vol__aseg__ab__lh_sum (and _rh_sum)**
   - **What it measures:** Total volume (mm³) of the left/right nucleus accumbens from FreeSurfer’s subcortical segmentation.
   - **Why it’s useful:** This is the classic structural metric—literally the size of the NAcc. It’s stable, easy to interpret, and visually clear when you explore the region in the ABCD Brain Atlas Visualizer.

2. **mr_y_smri__t1__aseg__ab__lh_mean (and _rh_mean)**
   - **What it measures:** Mean T1-weighted intensity within the NAcc (left/right).
   - **Why it’s useful:** This reflects tissue contrast and integrity. It isn’t a “size” measure but a signal measure related to microstructural properties (e.g., myelination, iron content, water density). It provides a complementary way of looking at NAcc structure beyond just volume.
---

### 2. **Extract NAcc Time-Series**  
You will extract the BOLD signal (functional MRI activation) from the NAcc across time. “Blood-Oxygen-Level-Dependent (BOLD) signal”: this a measure of blood flow to a brain region, which we use as a proxy for neural activity. 

**Why?**  
✅ To isolate the activity from our key brain region, so we can later connect its behavior to reward anticipation and feedback.

---

### 3. **Build Reward vs Neutral Arrays**  
You will separate the NAcc activation values into two groups: Reward trials vs Neutral trials.  
**Why?**  
✅ To directly compare how strongly the NAcc responds to rewarding cues versus neutral cues, helping test the incentive-salience hypothesis.  
🧠 The **incentive-salience hypothesis** proposes that dopamine transforms neutral cues into “wanted” signals, giving them motivational power to drive behavior.

---

### 4. **Visualizations & Simple Probabilistic Insights**  
You will generate several figures:  
- Box plot of means  
- Violin plot of full distributions  
- Empirical probability $P(\mathrm{Reward} > \mathrm{Neutral})$  
- Bootstrap confidence interval  
- Whole-brain activation maps  

**Why?**  
✅ Different plots let you see the data from multiple perspectives: central tendency, spread, uncertainty, and brain location.  
✅ Combining visual and statistical approaches builds a stronger, more trustworthy analysis.

---

Let’s get started! 🚀


In [ ]:
# ════════════════════════════════════════════════════════════════════
# Import Libraries & Configure Environment
# ════════════════════════════════════════════════════════════════════

# Core libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# fMRI analysis
from nilearn import image, masking
from nilearn.image import resample_to_img
from nilearn.masking import apply_mask

# Interactive widgets
from ipywidgets import interact
from IPython.display import Image, display

# Network utilities (for data download)
import urllib.request
import gzip
import shutil

# ────────────────────────────────────────────────────────────────────
# Configuration
# ────────────────────────────────────────────────────────────────────

# Display plots inline
%matplotlib inline

# Set visualization style
sns.set_theme(context="notebook", style="whitegrid", palette="muted", font_scale=1.2)

# Create output folder for figures
os.makedirs("figs", exist_ok=True)

print("✅ Environment configured successfully")

---
## Data Source, Citation & Folder Layout

**🚨 TEAMWORK NOTE (read carefully!)**  
You can work with any of the four available subjects:
- **Subject 01:** set `SUBJECT = "sub-s001"`  
- **Subject 02:** set `SUBJECT = "sub-s002"`  
- **Subject 03:** set `SUBJECT = "sub-s003"`  
- **Subject 04:** set `SUBJECT = "sub-s004"`  

In this exercise we use **real fMRI data** from the Adolescent Health and Development in Context (AHDC) study, publicly available on OpenNeuro:

> Baldwin M. Way, Christopher R. Browning, Dylan D. Wagner, Jodi L. Ford, Bethany Boettner & Ping Bai (2025).  
> _Structural and functional MRI dataset from the Adolescent Health and Development in Context (AHDC) study in Columbus, Ohio._  
> OpenNeuro [Dataset] doi:10.18112/openneuro.ds005901.v1.0.0  
> https://github.com/OpenNeuroDatasets/ds005901

This dataset was collected as part of a **longitudinal neuroimaging study of adolescent health and development**. Participants completed surveys, smartphone-based ecological momentary assessments, and MRI scans across multiple waves. One aim was to understand how **community exposures and reward processes** shape brain function and substance use risk.

We'll focus on the **Monetary Incentive Delay (MID)** task, which measures reward anticipation and outcome processing. You can choose to analyze any of the four subjects' data from the first MID run.

> **Before you begin, set `SUBJECT = "sub-s001"`, `"sub-s002"`, `"sub-s003"`, or `"sub-s004"`** in the code cell below.

Each subject's folder contains:  
- a 4D BOLD fMRI volume:  
  `sub-s00X_task-mid_run-01_bold.nii.gz`  
- its trial timing file:  
  `sub-s00X_task-mid_run-01_events.tsv`  
- (you'll also need the bilateral NAcc ROI mask: `nacc_bilateral_mask.nii`, placed in the `data/` folder)

### Download reliability note (important)

OpenNeuro’s “snapshot” download URLs are convenient for version pinning, but they may occasionally return transient **502 Bad Gateway** errors. This notebook’s download code uses a robust fallback order:

- **Primary (pinned):** OpenNeuro snapshot URL (uses `SNAPSHOT = "1.0.0"`)
- **Fallback:** Direct OpenNeuro S3 object URL:  
  `https://s3.amazonaws.com/openneuro.org/{DATASET}/{SUBJECT}/func/{bold_fname}`

We **do not** use GitHub raw URLs for the BOLD `.nii.gz` file because large OpenNeuro files in that repo are often stored via **git-annex**; the GitHub URL may return a small pointer file rather than real gzip data. (GitHub is still used as a fallback for the small `events.tsv`.)

**Expected directory structure (this repo):**

```
Exercise8/
├── Exercise8.ipynb ← This notebook
├── figs/ ← Saved figures will go here
└── data/
    ├── nacc_bilateral_mask.nii
    ├── qc_nacc_roi_alignment.png ← Static QC image (if provided)
    ├── sub-s001/
    ├── sub-s002/
    ├── sub-s003/
    └── sub-s004/
```

In [ ]:
# ──────────────────────────────────

# 0. Data folder constants

# ──────────────────────────────────

# Data will be stored in the 'data' subfolder relative to this notebook
# This works whether the notebook is opened directly or from the repo root
DATA_ROOT = Path("data")

print(f"📁 Data root: {DATA_ROOT.resolve()}")


## Select Your Subject

Choose which subject's data you'd like to analyze. Available subjects: sub-s001, sub-s002, sub-s003, sub-s004.

> **Set `SUBJECT = "sub‑s001"`, `"sub‑s002"`, `"sub‑s003"`, or `"sub‑s004"`** in the cell below.

In [ ]:
# Change this to "sub-s001", "sub-s002", "sub-s003", or "sub-s004" before you start!
SUBJECT = "sub-s001"

# Pre-defined file paths
FUNC_DIR = DATA_ROOT / SUBJECT / "func"
BOLD = FUNC_DIR / f"{SUBJECT}_task-mid_run-01_bold.nii"
BOLD_GZ = FUNC_DIR / f"{SUBJECT}_task-mid_run-01_bold.nii.gz"
EVENT = FUNC_DIR / f"{SUBJECT}_task-mid_run-01_events.tsv"
MASK = DATA_ROOT / "nacc_bilateral_mask.nii"

print("Selected subject:", SUBJECT)
print("  BOLD (uncompressed) →", BOLD)
print("  BOLD (compressed)   →", BOLD_GZ)
print("  EVENTS →", EVENT)
print("  MASK   →", MASK)

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Robust data download (OpenNeuro snapshot URL + direct S3 fallback; GitHub fallback for TSV only)
# ════════════════════════════════════════════════════════════════════

from pathlib import Path
import os
import shutil
import subprocess
import urllib.request
from urllib.error import HTTPError, URLError

DATASET  = "ds005901"
SNAPSHOT = "1.0.0"  # used by OpenNeuro snapshot URLs

# --------------------------------------------------------------------
# Helpers
# --------------------------------------------------------------------
def is_gzip_file(path: Path) -> bool:
    """Return True if file exists and has gzip magic bytes."""
    try:
        with open(path, "rb") as f:
            return f.read(2) == b"\x1f\x8b"
    except OSError:
        return False


def download_one(url: str, dest: Path) -> bool:
    """Download URL to dest, streaming in chunks. Returns True on success."""
    try:
        dest.parent.mkdir(parents=True, exist_ok=True)
        with urllib.request.urlopen(url) as r, open(dest, "wb") as f:
            shutil.copyfileobj(r, f, length=1024 * 1024)
        return True
    except (HTTPError, URLError) as e:
        print(f"   ⚠️  failed: {e}")
        return False
    except Exception as e:
        print(f"   ⚠️  failed: {type(e).__name__}: {e}")
        return False


def download_with_fallback(urls: list[str], dest: Path) -> Path | None:
    """Try URLs in order; returns dest if downloaded, else None."""
    for i, url in enumerate(urls, start=1):
        print(f"⬇️  [{i}/{len(urls)}] {dest.name} ← {url}")
        if download_one(url, dest):
            print("   …downloaded")
            return dest
    return None


def ensure_openneuro_py() -> None:
    """Install openneuro-py once per session if needed."""
    try:
        subprocess.run(
            ["openneuro-py", "--help"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True,
        )
    except Exception:
        subprocess.run(["python3", "-m", "pip", "install", "-q", "openneuro-py"], check=True)


def openneuro_py_download(relpath: str, target_dir: str) -> None:
    """
    relpath example: 'sub-s001/func/sub-s001_task-mid_run-01_bold.nii.gz'
    Downloads into target_dir/..., preserving the relpath structure.
    """
    ensure_openneuro_py()
    subprocess.run(
        [
            "openneuro-py",
            "download",
            f"--dataset={DATASET}",
            f"--target-dir={target_dir}",
            f"--include={relpath}",
        ],
        check=True,
    )


# --------------------------------------------------------------------
# Build paths + URLs
# --------------------------------------------------------------------
FUNC_DIR.mkdir(parents=True, exist_ok=True)

events_fname = f"{SUBJECT}_task-mid_run-01_events.tsv"
bold_fname   = f"{SUBJECT}_task-mid_run-01_bold.nii.gz"

evt_path     = FUNC_DIR / events_fname
bold_path_gz = FUNC_DIR / bold_fname

# OpenNeuro "snapshot" URLs (sometimes return 502 during outages)
openneuro_evt = (
    f"https://openneuro.org/crn/datasets/{DATASET}/snapshots/{SNAPSHOT}/files/"
    f"{SUBJECT}:func:{events_fname}?download=1"
 )
openneuro_bold = (
    f"https://openneuro.org/crn/datasets/{DATASET}/snapshots/{SNAPSHOT}/files/"
    f"{SUBJECT}:func:{bold_fname}?download=1"
 )

# Direct S3 URLs (often work even if OpenNeuro web/API is down)
s3_evt  = f"https://s3.amazonaws.com/openneuro.org/{DATASET}/{SUBJECT}/func/{events_fname}"
s3_bold = f"https://s3.amazonaws.com/openneuro.org/{DATASET}/{SUBJECT}/func/{bold_fname}"

# GitHub raw: OK for TSV; DO NOT use GitHub raw for .nii.gz (git-annex pointer)
github_evt = f"https://github.com/OpenNeuroDatasets/{DATASET}/raw/main/{SUBJECT}/func/{events_fname}"

events_urls = [openneuro_evt, s3_evt, github_evt]
bold_urls   = [openneuro_bold, s3_bold]

print(f"🔍 Checking data files for {SUBJECT}…")

# EVENTS
if evt_path.exists() and evt_path.stat().st_size > 0:
    print(f"✅ {events_fname} already exists, skipping download.")
    EVENT = evt_path
else:
    EVENT = download_with_fallback(events_urls, evt_path)

# BOLD
if bold_path_gz.exists() and is_gzip_file(bold_path_gz):
    print(f"✅ {bold_fname} already exists and is gzip, skipping download.")
    bold_gz = bold_path_gz
else:
    bold_gz = download_with_fallback(bold_urls, bold_path_gz)

# --------------------------------------------------------------------
# Download events.tsv
# --------------------------------------------------------------------
print(f"🔍 Checking data files for {SUBJECT}…")
EVENT = download_with_fallback(events_urls, evt_path)

# --------------------------------------------------------------------
# Download BOLD (large) + verify gzip
# --------------------------------------------------------------------
bold_gz = download_with_fallback(bold_urls, bold_path_gz)

# If somehow a non-gzip file landed at the destination, delete it
if bold_gz and not is_gzip_file(bold_gz):
    print("🧪 The downloaded BOLD file is not gzip. Deleting it.")
    try:
        os.remove(bold_gz)
    except OSError:
        pass
    bold_gz = None

# If direct URL routes failed, try openneuro-py (may also fail during OpenNeuro outages)
if bold_gz is None:
    try:
        print("🔁 Trying OpenNeuro via openneuro-py…")
        rel = f"{SUBJECT}/func/{bold_fname}"
        openneuro_py_download(rel, target_dir=str(DATA_ROOT))
        if bold_path_gz.exists() and is_gzip_file(bold_path_gz):
            bold_gz = bold_path_gz
        else:
            bold_gz = None
    except Exception as e:
        print(f"❌ openneuro-py download failed: {e}")
        bold_gz = None

BOLD = Path(bold_gz) if bold_gz else None

print("\n======================================================================")
print("📍 File paths:")
print(f"  BOLD:   {BOLD} (exists: {bool(BOLD and BOLD.exists())})")
print(f"  EVENTS: {EVENT} (exists: {bool(EVENT and EVENT.exists())})")
print(f"  MASK:   {MASK} (exists: {MASK.exists()})")

assert EVENT and EVENT.exists(), "Missing events.tsv after download."
assert BOLD and BOLD.exists(),  "Missing BOLD .nii.gz after download."

## 1. QC Anatomy & File Sanity Checks

Before analyzing brain activity, it's important to confirm that our data are correctly loaded and properly aligned. In the cell below we:

1. **Define file paths** to the subject's BOLD fMRI scan, the events timing file, and the NAcc region-of-interest (ROI) mask.  
2. **Check** that each file exists on your computer.  
3. **Compute** a mean image from the 4D BOLD volume to give a single, easy-to-view anatomical image.  
4. **Overlay** the ROI mask (in blue) and a marker for the Ventral Tegmental Area (VTA) (in yellow) to check alignment.

**What is the "mask"?**  
A **mask** in fMRI is a simplified image used to isolate specific brain regions for analysis. It acts like a stencil or cookie-cutter: white areas highlight the voxels to include (e.g., the nucleus accumbens), and black areas hide everything else. By checking the mask placement on the mean image, we ensure we're analyzing the correct brain structure before diving into the data.

This quality control step helps prevent misleading results caused by misaligned ROIs or missing files.

In [ ]:
# ════════════════════════════════════════════════════════════════════
# QC: file checks + generate a REAL NAcc ROI overlay on mean functional
# (replaces the two older cells)
# ════════════════════════════════════════════════════════════════════

import numpy as np
import nibabel as nib
from nilearn import image, plotting
from PIL import Image as PILImage
from IPython.display import display

print(f"📊 Analyzing: {SUBJECT}")
print("=" * 60)
print(f"  BOLD   exists? {BOLD.exists()}  -> {BOLD}")
print(f"  EVENTS exists? {EVENT.exists()} -> {EVENT}")
print(f"  MASK   exists? {MASK.exists()}  -> {MASK}")

# Stop early if anything required is missing
missing_files = []
if not BOLD.exists():
    missing_files.append(("BOLD", BOLD))
if not EVENT.exists():
    missing_files.append(("EVENTS", EVENT))
if not MASK.exists():
    missing_files.append(("MASK", MASK))

if missing_files:
    print("\n❌ Missing required files:")
    for label, path in missing_files:
        print(f"  - {label}: {path}")

    print("\nFixes:")
    print("  - Re-run the download cell above to fetch BOLD and EVENTS.")
    print("  - Confirm the instructor-provided MASK is present in the data folder.")
    raise FileNotFoundError("Required files missing; cannot generate QC figure.")

print("\n✅ All required files found. Generating a real QC overlay figure...")

# Helper: compute mask center-of-mass (mm) for sensible cut coordinates
def mask_center_mm(mask_path):
    m = nib.load(str(mask_path))
    data = m.get_fdata() > 0
    ijk = np.argwhere(data)
    if ijk.size == 0:
        return (0.0, 0.0, 0.0)
    com_ijk = ijk.mean(axis=0)
    com_xyz = nib.affines.apply_affine(m.affine, com_ijk)
    return (float(com_xyz[0]), float(com_xyz[1]), float(com_xyz[2]))

# Helper: parse task/run from filename for a clearer title
def parse_task_run(fname):
    task = None
    run = None
    for part in fname.split("_"):
        if part.startswith("task-"):
            task = part.replace("task-", "")
        if part.startswith("run-"):
            run = part.replace("run-", "")
    return task, run

task, run = parse_task_run(BOLD.name)
task_str = "MID task" if task == "mid" else (f"task-{task}" if task else "task")
run_str = f"run-{run}" if run else "run"

cut_coords = mask_center_mm(MASK)
qc_image_path = DATA_ROOT / "qc_nacc_roi_alignment.png"

# Always generate (overwrite) so the QC image is real, not a placeholder
mean_func = image.mean_img(str(BOLD))
disp = plotting.plot_roi(
    str(MASK),
    bg_img=mean_func,
    title=f"QC: Verify NAcc ROI alignment (mean fMRI, {SUBJECT}, {task_str}, {run_str})",
    display_mode="ortho",
    cut_coords=cut_coords,
    draw_cross=True,
)
disp.savefig(str(qc_image_path), dpi=250)
disp.close()

print(f"\n📍 QC figure saved to: {qc_image_path}")
print(f"   Cut coordinates (mm): x={cut_coords[0]:.1f}, y={cut_coords[1]:.1f}, z={cut_coords[2]:.1f}")
display(PILImage.open(qc_image_path))


### Interpreting the QC Image & Checklist

**Static QC Image shows:**
- **Anatomical Brain Slices:** Three orthogonal (coronal, sagittal, axial) views of the MNI brain template.
- **Crosshairs & Coordinates:** White crosshairs intersect at the approximate NAcc centre (MNI: –8, 0, 8).
- **Orientation:** “L” and “R” indicate left/right hemispheres in neurological convention.

**✅ QC Checklist - Verify before proceeding:**
1. All three files (BOLD, EVENT, MASK) exist and are accessible.  
2. The crosshairs sit over the ventral striatum region (just below the anterior commissure).  
3. The intersection point matches both left and right NAcc locations.  
4. The displayed coordinates make anatomical sense relative to known brain landmarks.  

> **Why this QC step matters:**  
> By confirming file accessibility and visually checking that the crosshairs are positioned in the ventral striatum, you ensure that the time-series you extract in the next step actually comes from the intended anatomical region. If the alignment or coordinates are off, all downstream comparisons (Reward vs Neutral) could be invalid.


## 2. Extract NAcc Time-Series

Before we dive in, let's simplify **TR** (Repetition Time):

> **TR = the interval between "snapshots."**  
> Every TR seconds (e.g. 2 s) the scanner takes one full 3D "photo" of your brain. When we want the BOLD signal right after an event, we convert the event time (in seconds) into which snapshot number (TR index) to pull.

In the cell below we use a helper function `extract_psc()` that:

1. **Loads** the 4D BOLD time-series image
2. **Resamples the ROI mask** to match the BOLD data's voxel grid and coordinate system  
   🧠 This ensures each mask voxel correctly overlaps with corresponding BOLD voxels
3. **Applies the mask** to extract BOLD values from NAcc voxels across all time points  
   🎯 Isolates the signal over time from just the NAcc—ignoring everything else in the brain
4. **Averages across voxels** at each TR to produce a single NAcc time-series  
5. **Converts to percent-signal-change (PSC)** for standardized, interpretable units

The function returns both:
- `nacc_psc`: percent-signal-change time-series (for analysis)
- `nacc_ts`: raw signal time-series (for reference)

In [ ]:
# ════════════════════════════════════════════════════════════════════
# 2. Extract NAcc time-series
# ════════════════════════════════════════════════════════════════════

def extract_psc(ts_img, mask_img):
    """
    Helper function to extract percent-signal-change time-series from an ROI.
    
    Parameters:
    -----------
    ts_img : str or Nifti-like
        Path to 4D BOLD time-series image
    mask_img : str or Nifti-like  
        Path to binary ROI mask
        
    Returns:
    --------
    psc_ts : numpy.ndarray
        1D array of percent-signal-change values (one per TR)
    raw_ts : numpy.ndarray
        1D array of raw signal values (one per TR)
    """
    # Load the 4D BOLD image
    bold_img = image.load_img(str(ts_img))
    
    # Resample mask to BOLD space (nearest-neighbor keeps it binary)
    mask_res = resample_to_img(
        source_img=str(mask_img),
        target_img=bold_img,
        interpolation="nearest"
    )
    
    # Extract time-series from all voxels in the mask
    # apply_mask returns array of shape (n_volumes, n_voxels)
    roi_vox_ts = apply_mask(bold_img, mask_res)
    
    # Average across voxels to get single ROI trace per TR
    # result is 1D array of length n_volumes
    raw_ts = roi_vox_ts.mean(axis=1)
    
    # Convert to percent-signal-change (PSC)
    baseline = raw_ts.mean()
    psc_ts = 100 * (raw_ts - baseline) / baseline
    
    print(f"Extracted time-series length: {len(psc_ts)} TRs")
    print(f"Baseline mean signal = {baseline:.1f}; converted to PSC.")
    
    return psc_ts, raw_ts

# 2a) Extract NAcc time-series using our helper function
nacc_psc, nacc_ts = extract_psc(BOLD, MASK)

# 2b) Quick peek: show first k rows of the PSC time-series in a table
df_nacc = pd.DataFrame(nacc_psc, columns=["psc_nacc"])

# interactive slider to choose how many rows to display
interact(
    lambda k: df_nacc.head(k),
    k=(1, min(20, len(df_nacc)), 1)  # slider from 1 up to 20 (or total TRs if fewer)
)

### What just happened?

- **Raw → PSC:** We calculated the average NAcc signal across the entire run (`baseline`), then converted each TR’s mean activation into **percent-signal-change** (`nacc_psc`). Now “0 %” means no change from baseline, and “+1 %” means a 1 % increase.
- **What is PSC?** **Percent-Signal-Change (PSC)** expresses each TR’s BOLD signal as a percentage change from its baseline, effectively measuring the percent change in blood flow (and thus neural activity) within your ROI (Brain Region of Interest).
- **Interactive preview:** The slider lets you inspect the first k rows of the PSC time-series. Try moving it between 1 and 20 (or more) to see how NAcc activation evolves over those snapshots.
- **Why PSC?** Converting to percent-signal-change puts all activation values on an intuitive, comparable scale—standard practice in fMRI—so your subsequent plots speak in “percent change” rather than arbitrary intensities.


## 3. Build Reward vs Neutral Arrays

In this step we:

- **Load** the MID events file and read each trial's onset time (in seconds) and `trial_type`.
- **Convert** each onset time to a TR index, then add a **4–5 TR lag** to capture the hemodynamic peak.
- **Average** the BOLD signal over the following **2 TRs** to get one activation value per trial.
- **Restrict to cue epochs** by filtering rows that start with `Cue_` — this isolates the **anticipation** period (aligns with ABCD MID ARvN betas).
- **Classify** `Cue_*Reward*` trials into `reward_vals` and `Cue_Triangle` trials into `neutral_vals`.

At the end you'll have two lists (`reward_vals` and `neutral_vals`) ready for plotting and probabilistic comparisons.

In [ ]:
# ════════════════════════════════════════════════════════════════════
# 3. Build Reward vs Neutral arrays (using PSC time-series)
# ════════════════════════════════════════════════════════════════════

# 3a) Load events.tsv
events = pd.read_csv(str(EVENT), sep='\t')
print(events[['onset', 'trial_type']].head())

# 3b) Get TR from header
TR = image.load_img(str(BOLD)).header.get_zooms()[3]
print(f"Repetition time (TR) = {TR} s")

# 3c) Define a helper to sample NAcc PSC after onset
def get_activation(ts, onset, lag_TRs=5, win_TRs=2, TR=1.0):
    """
    Return mean NAcc percent-signal-change in the window 
    [onset+lag, onset+lag+win) measured in TRs.
    """
    onset_TR = int(onset / TR)
    start = onset_TR + lag_TRs
    end = start + win_TRs
    return ts[start:end].mean()

# 3d) Restrict to cue epochs (anticipation). Build reward vs neutral lists
reward_vals = []
neutral_vals = []

cue_events = events[events['trial_type'].str.startswith('Cue')]

for _, trial in cue_events.iterrows():
    act = get_activation(
        nacc_psc,               # using percent‐signal‐change series
        trial['onset'],
        lag_TRs=5,
        win_TRs=2,
        TR=TR
    )
    if 'Reward' in trial['trial_type']:
        reward_vals.append(act)
    elif 'Triangle' in trial['trial_type']:
        neutral_vals.append(act)

# 3e) Quick sanity check
print(f"→ {len(reward_vals)} reward CUE trials")
print(f"→ {len(neutral_vals)} neutral (Triangle) CUE trials")

**What this output tells us**:

- The first five rows confirm we’ve correctly loaded the event file, showing each trial’s onset time (in seconds) and its type.
- The **Repetition time (TR) = 1.0 s** means the scanner collected one whole‐brain volume every second.
- We found **80 “Reward” trials** and **40 “Triangle” (neutral) trials**.  
  These counts become the lengths of our two lists—`reward_vals` (80 samples) and `neutral_vals` (40 samples)—which we’ll compare in the next steps.


### Quick Check: Compute Percent-Signal-Change Means

Now that we’ve converted our time-series to percent-signal-change (`nacc_psc`), let’s compute the average PSC for each condition.

🧠 **First try writing the code without AI (eg, GitHub Copilot)**—this helps you internalize the syntax.  
🤖 **If you get stuck**, let Copilot suggest a hint or fill in the blanks.

Below, fill in the two lines to calculate:

- `mean_reward`: the average PSC of all NAcc values on **Reward** trials  
- `mean_neutral`: the average PSC of all NAcc values on **Neutral (Triangle)** trials  

Finally, print their difference (`mean_reward - mean_neutral`) to see how much higher (or lower) the Reward response is compared to Neutral.


In [ ]:
# --- Student TO-DO: fill in the two lines below ---
mean_reward  = ...   # <- your code here
mean_neutral = ...   # <- your code here

print("Δ mean (Reward – Neutral) =", mean_reward - mean_neutral)

## 4. Visualisations

Below are the key plots you will generate.  Each cell saves its figure into `figs/` so you can easily pull them into your write‑up. The **Figs** folder is in the same folder as this file alongside our datasets.

### 4a) Event-Locked NAcc Time Course

Before comparing trial-averaged activations, let's visualize **when** the NAcc responds to cues by plotting the full time course around cue onset.

**What this plot shows:**
- **X-axis:** Time relative to cue presentation (seconds). Zero marks the moment the cue appears.
- **Y-axis:** NAcc percent-signal-change (PSC), averaged across all trials of each type.
- **Shaded bands:** Standard error across trials—wider bands indicate more variability between individual trials.

**Why this matters:**
- The **hemodynamic response** peaks ~5–7 seconds after a stimulus due to blood flow delays.
- By extracting a **peri-stimulus window** (2 TRs before to 14 TRs after cue onset), we capture the full rise, peak, and return-to-baseline of the BOLD signal.
- If the Reward curve (orange) rises higher than Neutral (blue) around 5–7 seconds post-cue, it confirms that **anticipation**—not just outcome—drives NAcc activation.

This temporal view complements the box plot (which collapses time) by showing the **dynamics** of reward processing.

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Build peri-stimulus time courses for Reward vs Neutral cues
# ════════════════════════════════════════════════════════════════════

win_pre, win_post = 2, 14   # TRs before/after cue
t = np.arange(-win_pre, win_post) * TR


def extract_epoch(ts, onset_s, TR, pre, post):
    onset_TR = int(onset_s / TR)
    idx = np.arange(onset_TR - pre, onset_TR + post)
    valid = (idx >= 0) & (idx < len(ts))
    out = np.full(idx.shape, np.nan, dtype=float)
    out[valid] = ts[idx[valid]]
    return out

cue_reward_onsets = cue_events[cue_events['trial_type'].str.contains('Reward')]['onset'].values
cue_neutral_onsets = cue_events[cue_events['trial_type'].str.contains('Triangle')]['onset'].values

epochs_reward = np.vstack([extract_epoch(nacc_psc, o, TR, win_pre, win_post) for o in cue_reward_onsets])
epochs_neutral = np.vstack([extract_epoch(nacc_psc, o, TR, win_pre, win_post) for o in cue_neutral_onsets])

m_reward = np.nanmean(epochs_reward, axis=0)
m_neutral = np.nanmean(epochs_neutral, axis=0)
se_reward = np.nanstd(epochs_reward, axis=0) / np.sqrt(np.sum(~np.isnan(epochs_reward), axis=0))
se_neutral = np.nanstd(epochs_neutral, axis=0) / np.sqrt(np.sum(~np.isnan(epochs_neutral), axis=0))

plt.figure(figsize=(7,4))
plt.plot(t, m_neutral, color="C0", label="Neutral cue")
plt.fill_between(t, m_neutral - se_neutral, m_neutral + se_neutral, color="C0", alpha=0.2)
plt.plot(t, m_reward, color="C1", label="Reward cue")
plt.fill_between(t, m_reward - se_reward, m_reward + se_reward, color="C1", alpha=0.2)
plt.axvline(0, color="k", linestyle=":", linewidth=1)
plt.xlabel("Time from cue (s)")
plt.ylabel("NAcc PSC (%)")
plt.title("Event-locked NAcc PSC around cue onset")
plt.legend()
plt.tight_layout()
plt.savefig("figs/anticipation_timecourse.png", dpi=150)
plt.show()

### Interpreting the Event-Locked Timecourse

**What to look for:**
1. **Baseline period (−2 to 0 s):** Both curves should hover near zero PSC before the cue appears—confirming our PSC baseline is valid.
2. **Rise phase (0 to ~5 s):** Both curves climb as blood flows to the NAcc in response to the cue, but does one rise faster or higher?
3. **Peak (~5–7 s):** The maximum PSC typically occurs here due to the hemodynamic lag. Look for separation between Reward and Neutral traces.
4. **Return to baseline (>10 s):** Both curves should descend back toward zero as the anticipation period ends.

**Key observations:**
- **Reward > Neutral during anticipation:** If the orange line stays consistently above blue during the 4–8 second window, it confirms that reward cues elicit **stronger anticipatory activation** in the NAcc—the neural signature of incentive salience.
- **Standard error bands:** Overlapping bands indicate high trial-to-trial variability, common in single-subject analyses. At the group level (many subjects), these bands typically separate more clearly.
- **Timing validates our lag_TRs choice:** The 5-TR lag we used for `get_activation()` samples the peak of this response curve, ensuring we capture maximum anticipation-related activity.

This plot bridges the gap between raw time-series (Section 2) and summary statistics (next sections), showing that the NAcc "wakes up" specifically when rewards are anticipated.

### Box Plot with Mean Lines

Here we’ll draw a box plot of Reward vs Neutral activations and overlay horizontal lines at the condition means you computed above (`mean_reward`, `mean_neutral`).


In [ ]:
# ════════════════════════════════════════════════════════════════════
# Box plot with mean lines
# ════════════════════════════════════════════════════════════════════

plt.figure(figsize=(6,4))

# 4a) boxplot
sns.boxplot(data=[neutral_vals, reward_vals],
            palette=["C0","C1"],
            showfliers=False)

# overlay the means
plt.hlines(mean_neutral, -0.4, 0.4, color="C0", linestyle="--",
           label=f"Neutral mean ({mean_neutral:.2f}%)")
plt.hlines(mean_reward,  0.6, 1.4,  color="C1", linestyle="--",
           label=f"Reward mean ({mean_reward:.2f}%)")

plt.xticks([0,1], ["Neutral cue","Reward cue"])
plt.ylabel("NAcc percent signal change (%)")
plt.title("4a) NAcc PSC by cue type (box + means)")
plt.legend(loc="upper left")

plt.savefig("figs/4a_box_means.png", dpi=150, bbox_inches="tight")
plt.show()

**Interpretation.**  
- The box represents the interquartile range (IQR) of activations; whiskers extend to ±1.5×IQR.  
- Dashed lines mark your previously computed means—now it’s clear how far Reward’s mean sits above Neutral’s.


### Bootstrap Confidence Interval & Effect Size

Now let's quantify the **uncertainty** and **magnitude** of the Reward vs Neutral difference using two complementary statistics:

**What this analysis does:**
1. **Bootstrap resampling (5,000 iterations):** Randomly samples with replacement from your reward and neutral trial data to simulate what would happen if you ran this experiment thousands of times. This generates a distribution of possible mean differences.
2. **95% Confidence Interval (CI):** The range within which we're 95% confident the true population mean difference falls. If this interval excludes zero, the effect is statistically significant.
3. **Cohen's d:** A standardized effect size that measures the difference in means relative to the pooled standard deviation. This tells us whether the difference is practically meaningful, not just statistically detectable.

**Why both metrics matter:**
- **CI answers:** "How certain are we about the direction and magnitude of the effect?"
- **Cohen's d answers:** "How large is this effect in real-world terms?"

**Interpreting the output:**

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Bootstrap CI and Cohen's d effect size summary
# ════════════════════════════════════════════════════════════════════

rng = np.random.default_rng(42)
B = 5000
boot_diff = []
for _ in range(B):
    r = rng.choice(reward_vals, size=len(reward_vals), replace=True)
    n = rng.choice(neutral_vals, size=len(neutral_vals), replace=True)
    boot_diff.append(r.mean() - n.mean())
ci_lo, ci_hi = np.percentile(boot_diff, [2.5, 97.5])

def cohens_d(a, b):
    a, b = np.asarray(a), np.asarray(b)
    s1, s2 = a.std(ddof=1), b.std(ddof=1)
    sp = np.sqrt(((len(a)-1)*s1**2 + (len(b)-1)*s2**2) / (len(a)+len(b)-2))
    return (a.mean() - b.mean()) / sp

d = cohens_d(reward_vals, neutral_vals)
print(f"Δ mean = {mean_reward - mean_neutral:.3f}%  (95% CI [{ci_lo:.3f}, {ci_hi:.3f}]),  Cohen's d = {d:.2f}")

### Interpreting Bootstrap CI & Cohen's d

**Understanding the printed statistics:**

| Statistic | Interpretation |
|-----------|----------------|
| **Δ mean = X.XX%** | Average difference in NAcc PSC between Reward and Neutral cues across all trials. Positive = Reward higher. |
| **95% CI [lo, hi]** | We're 95% confident the true mean difference lies in this range. If both bounds are positive (> 0), Reward significantly exceeds Neutral. |
| **Cohen's d** | Standardized effect size:<br>• **d < 0.2:** Trivial effect<br>• **0.2–0.5:** Small effect<br>• **0.5–0.8:** Medium effect<br>• **d > 0.8:** Large effect |

**What this tells us:**
- **If CI excludes zero (e.g., [0.05, 0.31]):** The reward anticipation effect is **statistically reliable**—even accounting for trial-to-trial noise, we'd expect Reward cues to consistently activate NAcc more than Neutral cues.
- **If d ≈ 0.3–0.5:** The effect is **small to medium** in magnitude—detectable, but subject to individual differences and measurement noise (typical for single-subject fMRI).
- **If d > 0.5:** The effect is **moderate to large**—reward cues drive a substantial difference in NAcc activation, supporting the incentive-salience hypothesis even at the individual level.

**Connecting to neuroscience:**
This quantitative summary bridges the visual patterns (timecourse, box plot) and the underlying biology: a Cohen's d of 0.4–0.6 suggests that dopamine-driven anticipation produces a **moderate but consistent shift** in NAcc activity—exactly what we'd predict if the brain is learning to "want" reward-predicting cues more over time.

---

## 5 · Key Take-Aways    

### What did we see in the MID task?

**Main finding:** The NAcc showed **higher activation for Reward vs Neutral cues** during the anticipation window.

| Evidence | What it means |
|----------|---------------|
| **Event-locked timecourse** | Reward cues (orange) peaked higher than Neutral cues (blue) ~5–7 seconds post-cue—exactly when anticipation is strongest. |
| **Box plot + means** | Reward trials showed consistently higher PSC, with the mean difference captured by the dashed lines. |
| **Bootstrap CI & Cohen's d** | The 95% CI excluded zero (statistically reliable effect), and Cohen's d quantified a small-to-medium effect size typical of single-subject fMRI. |

**In plain language:** When the "win-money" cue appeared, the nucleus accumbens—the brain's "want-it" hub—showed a larger percent-signal-change than when the neutral cue appeared. This is the neural signature of **incentive salience**.

---

### How does this link back to *BuildingABrain.ipynb*?

Both your toy neural network and the human brain use the same three-step loop to learn from rewards:

| Step | Artificial Network | Human Brain |
|------|-------------------|-------------|
| **1. Teaching signal** | Computes prediction error after each guess → updates weights | VTA dopamine burst when outcome > expectation → modulates synapses |
| **2. Value learning** | Weights strengthen toward rewarded choices across training epochs | Cortico-striatal synapses strengthen with repeated reward → NAcc "wants" cue more |
| **3. Behavioral output** | Network selects higher-valued class at test time | Elevated NAcc activity drives faster responses when money is at stake |

---

### Why does it matter?

**Mechanistic insight:**  
Seeing NAcc PSC rise after reward cues provides a concrete, measurable readout of the abstract "prediction-error learning" you coded in BuildingABrain. The same computational principles—**error signals**, **weight updates**, **learned associations**—operate in both silicon and neurons.

**Clinical relevance:**  
If dopamine bursts are hijacked (e.g., by addictive drugs or maladaptive cues), this same learning loop can over-train the NAcc, inflating "wanting" for harmful stimuli. Understanding the mechanism is the first step toward interventions that re-wire these associations.

---

**Take a moment to look back at your three plots—each one is a puzzle piece showing *when* (timecourse), *how much* (box plot), and *how certain* (bootstrap CI) the NAcc responds to anticipated rewards.**

---